In [56]:
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import csv
import numpy as np

lags = 4
edges = np.array([[0, 2], [0, 4], [1, 2], [1, 4], [3, 4]]).T
edge_weights = np.ones(edges.shape[1])

data_array = []
with open('top_5_373.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    for row in csv_reader:
        data_row = [float(value) for value in row[1:]]
        data_array.append(data_row)

data_2d_array = np.array(data_array)
# print(data_2d_array)
features = [
    data_2d_array[i : i + lags, :].T
    for i in range(data_2d_array.shape[0] - lags)
]
targets = [
    data_2d_array[i + lags, :].T
    for i in range(data_2d_array.shape[0] - lags)
]

dataset = StaticGraphTemporalSignal(edges, edge_weights, features, targets)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)

# print(features)


In [57]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [59]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 200/200 [02:18<00:00,  1.44it/s]


In [62]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    print(y_hat)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

tensor([[4.5227],
        [4.4925],
        [4.4535],
        [4.5225],
        [3.9320]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5226],
        [4.4961],
        [4.5224],
        [3.9242]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5227],
        [4.5106],
        [4.5221],
        [4.2806]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5227],
        [4.5131],
        [4.5206],
        [4.3641]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5227],
        [4.4874],
        [4.5010],
        [4.2673]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5227],
        [4.3400],
        [4.4451],
        [4.2499]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5225],
        [4.1305],
        [4.4140],
        [3.8964]], grad_fn=<AddmmBackward0>)
tensor([[4.5227],
        [4.5223],
        [3.9242],
        [4.4225],
        [4.1526]], grad_fn=<AddmmBackward0>)
tensor([[4.5226],
        [4.5185],
        [3.9080],
        [4

In [1]:
import pandas as pd

df = pd.read_csv("top_5_373.csv")

In [5]:
print(df['46670'].mean())

7.025445292620865
